In [1]:
# load required libraries & modules
%load_ext autoreload
%autoreload 2

import os
from tqdm.notebook import tqdm
import pprint
import time
import warnings
# warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import torch

from utils import *
from loaddata import *
from visualization_original import *
from rrcapsnet_original import *

torch.set_grad_enabled(False)
torch.set_printoptions(sci_mode=False)

DATA_DIR = '../data'
DEVICE = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
# DEVICE = torch.device('cpu')

BATCHSIZE = 1000

PATH_MNISTC = '../data/MNIST_C/'
CORRUPTION_TYPES = ['identity', 
         'shot_noise', 'impulse_noise','glass_blur','motion_blur',
         'shear', 'scale',  'rotate',  'brightness',  'translate',
         'stripe', 'fog','spatter','dotted_line', 'zigzag',
         'canny_edges']

N_MINI_PER_CORRUPTION = 1000

# general helper funtions for model testing
def load_model(args):
    # load model
    model = RRCapsNet(args).to(args.device) 
    model.load_state_dict(torch.load(args.load_model_path))
    return model

def load_args(load_model_path, args_to_update, verbose=False):
    params_filename = os.path.dirname(load_model_path) + '/params.txt'
    assert os.path.isfile(params_filename), "No param flie exists"
    args = parse_params_wremove(params_filename, removelist = ['device']) 
    args = update_args(args, args_to_update)
    args.load_model_path = load_model_path
    if verbose:
        pprint.pprint(args.__dict__, sort_dicts=False)
    return args

def test_model(task, model, args, verbose=False):
    # set task and print setting
    if verbose:
        pprint.pprint(args.__dict__, sort_dicts=False)
    
    # get test results
    model.eval()
    test_dataloader = fetch_dataloader(task, DATA_DIR, DEVICE, BATCHSIZE, train=False)
    test_loss, test_loss_class, test_loss_recon, test_acc = test(model, test_dataloader, args, acc_name="dynamic")
    if verbose:
        print("==> test_loss=%.5f, test_loss_class=%.5f, test_loss_recon=%.5f, test_acc=%.4f"
              % (test_loss, test_loss_class, test_loss_recon, test_acc))
    return test_loss, test_loss_class, test_loss_recon, test_acc

# helper funtions for model testing on mnist-C
def test_model_mnistc(path_mnistc, corruptionlist, model, verbose=False):
    # set task and print setting
    if verbose:
        pprint.pprint(args.__dict__, sort_dicts=False)
    
    # get average test results over corruptionlist   
    losses, classlosses, reconlosses, accs = [], [], [], []
    for corruption in corruptionlist:
        test_loss, test_loss_class, test_loss_recon, test_acc = test_model_on_each_corruption(path_mnistc, corruption, model, verbose)

        losses.append(test_loss)
        classlosses.append(test_loss_class)
        reconlosses.append(test_loss_recon)
        accs.append(test_acc)
    
    avgtest_loss = sum(losses)/len(corruptionlist)
    avgtest_loss_class = sum(classlosses)/len(corruptionlist)
    avgtest_loss_recon = sum(reconlosses)/len(corruptionlist)
    avgtest_acc = sum(accs)/len(corruptionlist)
    
    if verbose:
        print("==> average test_loss=%.5f, test_loss_class=%.5f, test_loss_recon=%.5f, test_acc=%.4f"
              % (avgtest_loss, avgtest_loss_class, avgtest_loss_recon, avgtest_acc))
        
    return avgtest_loss, avgtest_loss_class, avgtest_loss_recon, avgtest_acc
    
def test_model_on_each_corruption(path_mnistc, corruption, model, verbose=False):
    path_images = os.path.join(path_mnistc, corruption, 'test_images.npy')
    path_labels = os.path.join(path_mnistc, corruption, 'test_labels.npy')

    # convert to torch
    images = np.load(path_images)
    labels = np.load(path_labels)
    transform_tohot = T.Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
    images_tensorized = torch.stack([T.ToTensor()(im) for im in images])
    labels_tensorized = torch.stack([transform_tohot(label) for label in labels])
    # print(images_tensorized.shape) #torch.Size([10000, 1, 28, 28])
    # print(labels_tensorized.shape) #torch.Size([10000, 10])

    # create dataloader
    kwargs = {'num_workers': 1, 'pin_memory': True} if DEVICE == 'cuda' else {}
    dataset = TensorDataset(images_tensorized, labels_tensorized)
    dataloader = DataLoader(dataset, batch_size=BATCHSIZE, shuffle=False, drop_last=False, **kwargs)

    # test on the dataloder
    model.eval()
    test_loss, test_loss_class, test_loss_recon, test_acc = test(model, dataloader, args, acc_name="dynamic")

    if verbose:
        print("==> individual test_loss=%.5f, test_loss_class=%.5f, test_loss_recon=%.5f, test_acc=%.4f"
              % (test_loss, test_loss_class, test_loss_recon, test_acc))
    return test_loss, test_loss_class, test_loss_recon, test_acc



@torch.no_grad()
def evaluate_on_batch(task, batchnum, model, args, train=False, verbose=True, onlyacc=False):
    
    # evaluate on one train/test batch 
    model.eval()
    
    if task == 'mnist_recon':
        # for mnist recon data, it has erased input(x) and intact input (gtx)
        if train:
            dataloader, val_dataloader = fetch_dataloader(task, DATA_DIR, DEVICE, BATCHSIZE, train)
        else:
            dataloader = fetch_dataloader(task, DATA_DIR, DEVICE, BATCHSIZE, train)            
        diter = iter(dataloader)
        for i in range(batchnum):
            x, gtx, y = next(diter)
    else:
        dataloader = fetch_dataloader(task, DATA_DIR, DEVICE, BATCHSIZE, train)    
        diter = iter(dataloader)
        for i in range(batchnum):
            x, y = next(diter)
            gtx = None
            
            
    # attach forward hooks for intermediate outputs for visualizations
    outputs = {}
    
    # from model main output
    x_input_step = []; x_mask_step = []; objcaps_step = []
#     affine_params_step=[]
    
    # from model dynamic routing
    coups_step = []; betas_step= []; rscores_step=[]; recon_coups_step=[] 
    outcaps_len_step=[]; outcaps_len_before_step=[]

    def get_attention_outputs():
        def hook(model, input, output):
            x_mask_step.append(output[0].detach())
            x_input_step.append(output[1].detach())
        return hook

    def get_capsule_outputs():
        def hook(model, input, output):
            objcaps_step.append(output[0].detach())
            coups_step.append(torch.stack(output[1]['coups'], dim=1))
            betas_step.append(torch.stack(output[1]['betas'], dim=1)) 
            if 'rscores' in output[1].keys():
                rscores_step.append(torch.stack(output[1]['rscores'], dim=1))
            if 'recon_coups' in output[1].keys():
                recon_coups_step.append(torch.stack(output[1]['recon_coups'], dim=1))
            if 'outcaps_len' in output[1].keys():
                outcaps_len_step.append(torch.stack(output[1]['outcaps_len'], dim=1))
            if 'outcaps_len_before' in output[1].keys():
                outcaps_len_before_step.append(torch.stack(output[1]['outcaps_len_before'], dim=1))
        return hook
    

    hook1 = model.input_window.register_forward_hook(get_attention_outputs())
    hook2 = model.capsule_routing.register_forward_hook(get_capsule_outputs())
    
    # evaluate and detach hooks
    losses, acc, objcaps_len_step, x_recon_step = evaluate(model, x, y, args, acc_name='dynamic', gtx=gtx)
    hook1.remove()
    hook2.remove()
    
    # add tensor outputs dictionary
    outputs['x_input'] = torch.stack(x_input_step, dim=1)
    outputs['x_mask'] = torch.stack(x_mask_step, dim=1)
    outputs['objcaps'] = torch.stack(objcaps_step, dim=1)
    
    outputs['coups'] = torch.stack(coups_step, dim=1)
    outputs['betas'] = torch.stack(betas_step, dim=1)
    if rscores_step:
        outputs['rscores'] = torch.stack(rscores_step, dim=1)
    if recon_coups_step:
        outputs['recon_coups'] = torch.stack(recon_coups_step, dim=1)
    if outcaps_len_step:
        outputs['outcaps_len'] = torch.stack(outcaps_len_step, dim=1)
    if outcaps_len_before_step:
        outputs['outcaps_len_before'] = torch.stack(outcaps_len_before_step, dim=1)        
    if verbose:
        print("==> On this sigle test batch: test_loss=%.5f, test_loss_class=%.5f, test_loss_recon=%.5f, test_acc=%.4f"
              % (losses[0], losses[1], losses[2], acc))
    
    if onlyacc:
        return acc
    else:
        return x, gtx, y, objcaps_len_step, x_recon_step, outputs
    
    
@torch.no_grad()
def evaluate_model_on_mnistc_mini(corruption, model, args, train=False, verbose=True, save_hooks=False, max_batch_num=None):
    
    # get corruption batch information
    corruption_id = int(CORRUPTION_TYPES.index(corruption))
    num_batch_required = int(N_MINI_PER_CORRUPTION/BATCHSIZE) # if batchsize 100; 10 batches are requried
    
    # load dataloader and iterator
    dataloader = fetch_dataloader('mnist_c_mini', DATA_DIR, DEVICE, BATCHSIZE, train)    
    diter = iter(dataloader)

    if save_hooks:
        def get_attention_outputs():
            def hook(model, input, output):
                x_mask_step.append(output[0].detach())
                x_input_step.append(output[1].detach())
            return hook

        def get_capsule_outputs():
            def hook(model, input, output):
                objcaps_step.append(output[0].detach())
                coups_step.append(torch.stack(output[1]['coups'], dim=1))
                betas_step.append(torch.stack(output[1]['betas'], dim=1)) 
                if 'rscores' in output[1].keys():
                    rscores_step.append(torch.stack(output[1]['rscores'], dim=1))
                if 'recon_coups' in output[1].keys():
                    recon_coups_step.append(torch.stack(output[1]['recon_coups'], dim=1))
                if 'outcaps_len' in output[1].keys():
                    outcaps_len_step.append(torch.stack(output[1]['outcaps_len'], dim=1))
                if 'outcaps_len_before' in output[1].keys():
                    outcaps_len_before_step.append(torch.stack(output[1]['outcaps_len_before'], dim=1))
            return hook
        
        outputs = {}

        x_input_step_all = []; x_mask_step_all = []; objcaps_step_all = []

        coups_step_all = []; betas_step_all= []; rscores_step_all=[]; recon_coups_step_all=[] 
        outcaps_len_step_all=[]; outcaps_len_before_step_all=[]

    x_all, y_all, gtx_all, loss_all, acc_all, objcaps_len_step_all, x_recon_step_all = [],[],[],[],[],[],[]
    model.eval()        
    # get input and gt
    for i in range(corruption_id*num_batch_required): #id =0, 0 iteration; id=1, 10 iteration
        x, y = next(diter)
        gtx = None
    
    for i in range(0, num_batch_required):
        x, y = next(diter)
        gtx = None
        
        if max_batch_num:
            if i == max_batch_num:
                break
            
        # for hooks over other model output
        x_input_step = []; x_mask_step = []; objcaps_step = []

        if save_hooks:

            # for hooks over dynamic routing
            coups_step = []; betas_step= []; rscores_step=[]; recon_coups_step=[] 
            outcaps_len_step=[]; outcaps_len_before_step=[]

            hook1 = model.input_window.register_forward_hook(get_attention_outputs())
            hook2 = model.capsule_routing.register_forward_hook(get_capsule_outputs())

        # evaluate and append results 
        losses, acc, objcaps_len_step, x_recon_step = evaluate(model, x, y, args, acc_name='dynamic', gtx=gtx)
        
        if verbose:
            print("==> On this sigle test batch: test_loss=%.5f, test_loss_class=%.5f, test_loss_recon=%.5f, test_acc=%.4f"
                  % (losses[0], losses[1], losses[2], acc))   
            
        # main input and output append
        x_all.append(x)
        y_all.append(y)
        if gtx:
            gtx_all.append(gtx)
        #         loss_all.append(losses[0])
        acc_all.append(acc)
        objcaps_len_step_all.append(objcaps_len_step)
        x_recon_step_all.append(x_recon_step)
        
        if save_hooks:
        
            # hook variables append
            x_input_step_all.append(torch.stack(x_input_step, dim=1))
            x_mask_step_all.append(torch.stack(x_mask_step, dim=1))
            objcaps_step_all.append(torch.stack(objcaps_step, dim=1))

            coups_step_all.append(torch.stack(coups_step, dim=1))
            betas_step_all.append(torch.stack(betas_step, dim=1))
            if rscores_step:
                rscores_step_all.append(torch.stack(rscores_step, dim=1))
            if recon_coups_step:
                recon_coups_step_all.append(torch.stack(recon_coups_step, dim=1))
            if outcaps_len_step:
                outcaps_len_step_all.append(torch.stack(outcaps_len_step, dim=1))
            if outcaps_len_before_step:
                outcaps_len_before_step_all.append(torch.stack(outcaps_len_before_step, dim=1))

            hook1.remove()
            hook2.remove()
    
        
    # concat and add to outputs dictionary
    x_all = torch.cat(x_all, dim=0)
    y_all = torch.cat(y_all, dim=0)
    if gtx:
        gtx_all = torch.cat(gtx_all, dim=0)
    else:
        gtx_all = gtx
    acc_all = torch.cat(acc_all, dim=0)
    objcaps_len_step_all = torch.cat(objcaps_len_step_all, dim=0)
    x_recon_step_all = torch.cat(x_recon_step_all, dim=0)
    
    if save_hooks:
        outputs['x_input']= torch.cat(x_input_step_all, dim=0)
        outputs['x_mask']= torch.cat(x_mask_step_all, dim=0)
        outputs['objcaps']= torch.cat(objcaps_step_all, dim=0)

        outputs['coups'] = torch.cat(coups_step_all, dim=0)
        outputs['betas'] = torch.cat(betas_step_all, dim=0)
        if rscores_step_all:
            outputs['rscores'] = torch.cat(rscores_step_all, dim=0)
        if recon_coups_step_all:
            outputs['recon_coups'] = torch.cat(recon_coups_step_all, dim=0)
        if outcaps_len_step_all:
            outputs['outcaps_len'] = torch.cat(outcaps_len_step_all, dim=0)
        if outcaps_len_before_step_all:
            outputs['outcaps_len_before'] = torch.cat(outcaps_len_before_step_all, dim=0)
            
        return x_all, gtx_all, y_all, acc_all, objcaps_len_step_all, x_recon_step_all, outputs  

    else:
        return x_all, gtx_all, y_all, acc_all, objcaps_len_step_all, x_recon_step_all
    
    
@torch.no_grad()
def evaluate_cnn_on_mnistc_mini(corruption, cnn, max_batch_num=None):
    # get corruption batch information
    corruption_id = int(CORRUPTION_TYPES.index(corruption))
    num_batch_required = int(N_MINI_PER_CORRUPTION/BATCHSIZE) # if batchsize 100; 10 batches are requried
    
    # load dataloader and iterator
    dataloader = fetch_dataloader('mnist_c_mini', DATA_DIR, DEVICE, BATCHSIZE, train=False)    
    diter = iter(dataloader)
    
    # save output
    x_all, y_all, pred_all, acc_all, class_prob_all = [],[],[], [],[]
    cnn.eval() 

    # get input and gt
    for i in range(corruption_id*num_batch_required): #id =0, 0 iteration; id=1, 10 iteration
        x, y = next(diter)
    

    for i in range(0, num_batch_required):
        x, y = next(diter)
        gtx = None
        
        if max_batch_num:
            if i == max_batch_num:
                break
        
        data, target = x.to(DEVICE),  y.to(DEVICE)
        target = target.argmax(dim=1, keepdim=True)
        output = cnn(data)
        #                 test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        acc = pred.eq(target.view_as(pred))
        
        x_all.append(data)
        y_all.append(target.flatten())
        pred_all.append(pred.flatten())
        acc_all.append(acc.flatten().float())
        class_prob_all.append(output)
    

    # concat and add to outputs dictionary
    x_all = torch.cat(x_all, dim=0)
    y_all = torch.cat(y_all, dim=0)
    pred_all = torch.cat(pred_all, dim=0)
    acc_all = torch.cat(acc_all, dim=0)
    class_prob_all = torch.cat(class_prob_all, dim=0)

    return x_all, y_all, class_prob_all, pred_all, acc_all



    
    
@torch.no_grad()
def evaluate_model_on_mnistc_original(corruption, model, verbose=False, save_hooks=False):
    path_images = os.path.join(PATH_MNISTC, corruption, 'test_images.npy')
    path_labels = os.path.join(PATH_MNISTC, corruption, 'test_labels.npy')

    # convert to torch
    images = np.load(path_images)
    labels = np.load(path_labels)
    transform_tohot = T.Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
    images_tensorized = torch.stack([T.ToTensor()(im) for im in images])
    labels_tensorized = torch.stack([transform_tohot(label) for label in labels])
    # print(images_tensorized.shape) #torch.Size([10000, 1, 28, 28])
    # print(labels_tensorized.shape) #torch.Size([10000, 10])

    # create dataloader
    kwargs = {'num_workers': 1, 'pin_memory': True} if DEVICE == 'cuda' else {}
    dataset = TensorDataset(images_tensorized, labels_tensorized)
    dataloader = DataLoader(dataset, batch_size=BATCHSIZE, shuffle=False, drop_last=False, **kwargs)

    if save_hooks:
        def get_attention_outputs():
            def hook(model, input, output):
                x_mask_step.append(output[0].detach())
                x_input_step.append(output[1].detach())
            return hook

        def get_capsule_outputs():
            def hook(model, input, output):
                objcaps_step.append(output[0].detach())
                coups_step.append(torch.stack(output[1]['coups'], dim=1))
                betas_step.append(torch.stack(output[1]['betas'], dim=1)) 
                if 'rscores' in output[1].keys():
                    rscores_step.append(torch.stack(output[1]['rscores'], dim=1))
                if 'recon_coups' in output[1].keys():
                    recon_coups_step.append(torch.stack(output[1]['recon_coups'], dim=1))
                if 'outcaps_len' in output[1].keys():
                    outcaps_len_step.append(torch.stack(output[1]['outcaps_len'], dim=1))
                if 'outcaps_len_before' in output[1].keys():
                    outcaps_len_before_step.append(torch.stack(output[1]['outcaps_len_before'], dim=1))
            return hook
        
        outputs = {}

        x_input_step_all = []; x_mask_step_all = []; objcaps_step_all = []

        coups_step_all = []; betas_step_all= []; rscores_step_all=[]; recon_coups_step_all=[] 
        outcaps_len_step_all=[]; outcaps_len_before_step_all=[]

    x_all, y_all, gtx_all, loss_all, acc_all, objcaps_len_step_all, x_recon_step_all = [],[],[],[],[],[],[]
    
    model.eval()        
    # get input and gt
    for data in dataloader:
        x, y = data
        gtx = None
                    
        # for hooks over other model output
        x_input_step = []; x_mask_step = []; objcaps_step = []

        if save_hooks:

            # for hooks over dynamic routing
            coups_step = []; betas_step= []; rscores_step=[]; recon_coups_step=[] 
            outcaps_len_step=[]; outcaps_len_before_step=[]

            hook1 = model.input_window.register_forward_hook(get_attention_outputs())
            hook2 = model.capsule_routing.register_forward_hook(get_capsule_outputs())

        # evaluate and append results 
        losses, acc, objcaps_len_step, x_recon_step = evaluate(model, x, y, args, acc_name='dynamic', gtx=gtx)
        
        if verbose:
            print("==> On this sigle test batch: test_loss=%.5f, test_loss_class=%.5f, test_loss_recon=%.5f, test_acc=%.4f"
                  % (losses[0], losses[1], losses[2], acc))   
            
        # main input and output append
        x_all.append(x)
        y_all.append(y)
        if gtx:
            gtx_all.append(gtx)
        #         loss_all.append(losses[0])
        acc_all.append(acc)
        objcaps_len_step_all.append(objcaps_len_step)
        x_recon_step_all.append(x_recon_step)
        
        if save_hooks:
        
            # hook variables append
            x_input_step_all.append(torch.stack(x_input_step, dim=1))
            x_mask_step_all.append(torch.stack(x_mask_step, dim=1))
            objcaps_step_all.append(torch.stack(objcaps_step, dim=1))

            coups_step_all.append(torch.stack(coups_step, dim=1))
            betas_step_all.append(torch.stack(betas_step, dim=1))
            if rscores_step:
                rscores_step_all.append(torch.stack(rscores_step, dim=1))
            if recon_coups_step:
                recon_coups_step_all.append(torch.stack(recon_coups_step, dim=1))
            if outcaps_len_step:
                outcaps_len_step_all.append(torch.stack(outcaps_len_step, dim=1))
            if outcaps_len_before_step:
                outcaps_len_before_step_all.append(torch.stack(outcaps_len_before_step, dim=1))

            hook1.remove()
            hook2.remove()
    
        
    # concat and add to outputs dictionary
    x_all = torch.cat(x_all, dim=0)
    y_all = torch.cat(y_all, dim=0)
    if gtx:
        gtx_all = torch.cat(gtx_all, dim=0)
    else:
        gtx_all = gtx
    acc_all = torch.cat(acc_all, dim=0)
    objcaps_len_step_all = torch.cat(objcaps_len_step_all, dim=0)
    x_recon_step_all = torch.cat(x_recon_step_all, dim=0)
    
    if save_hooks:
        outputs['x_input']= torch.cat(x_input_step_all, dim=0)
        outputs['x_mask']= torch.cat(x_mask_step_all, dim=0)
        outputs['objcaps']= torch.cat(objcaps_step_all, dim=0)

        outputs['coups'] = torch.cat(coups_step_all, dim=0)
        outputs['betas'] = torch.cat(betas_step_all, dim=0)
        if rscores_step_all:
            outputs['rscores'] = torch.cat(rscores_step_all, dim=0)
        if recon_coups_step_all:
            outputs['recon_coups'] = torch.cat(recon_coups_step_all, dim=0)
        if outcaps_len_step_all:
            outputs['outcaps_len'] = torch.cat(outcaps_len_step_all, dim=0)
        if outcaps_len_before_step_all:
            outputs['outcaps_len_before'] = torch.cat(outcaps_len_before_step_all, dim=0)
            
        return x_all, gtx_all, y_all, acc_all, objcaps_len_step_all, x_recon_step_all, outputs  

    else:
        return x_all, gtx_all, y_all, acc_all, objcaps_len_step_all, x_recon_step_all
    
    

@torch.no_grad()
def evaluate_cnn_on_mnistc_original(corruption, cnn):
    path_images = os.path.join(PATH_MNISTC, corruption, 'test_images.npy')
    path_labels = os.path.join(PATH_MNISTC, corruption, 'test_labels.npy')

    # convert to torch
    images = np.load(path_images)
    labels = np.load(path_labels)
    transform_tohot = T.Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
    images_tensorized = torch.stack([T.ToTensor()(im) for im in images])
    labels_tensorized = torch.stack([transform_tohot(label) for label in labels])
    # print(images_tensorized.shape) #torch.Size([10000, 1, 28, 28])
    # print(labels_tensorized.shape) #torch.Size([10000, 10])

    # create dataloader
    kwargs = {'num_workers': 1, 'pin_memory': True} if DEVICE == 'cuda' else {}
    dataset = TensorDataset(images_tensorized, labels_tensorized)
    dataloader = DataLoader(dataset, batch_size=BATCHSIZE, shuffle=False, drop_last=False, **kwargs)

    # save output
    x_all, y_all, pred_all, acc_all, class_prob_all = [],[],[], [],[]
    cnn.eval() 

    # get input and gt

    for data in dataloader:
        x, y = data
        gtx = None
        
        data, target = x.to(DEVICE),  y.to(DEVICE)
        target = target.argmax(dim=1, keepdim=True)
        output = cnn(data)
        #                 test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        acc = pred.eq(target.view_as(pred))
        
        x_all.append(data)
        y_all.append(target.flatten())
        pred_all.append(pred.flatten())
        acc_all.append(acc.flatten().float())
        class_prob_all.append(output)
    

    # concat and add to outputs dictionary
    x_all = torch.cat(x_all, dim=0)
    y_all = torch.cat(y_all, dim=0)
    pred_all = torch.cat(pred_all, dim=0)
    acc_all = torch.cat(acc_all, dim=0)
    class_prob_all = torch.cat(class_prob_all, dim=0)

    return x_all, y_all, class_prob_all, pred_all, acc_all


import matplotlib
import collections
import json
import random

def save_imgarr(imgarr, filename='test.png', scale=8):
    h, w, _ = imgarr.shape
    fig, axes = plt.subplots(figsize=(h*scale, w*scale))
    fig.subplots_adjust(top=1.0, bottom=0, right=1.0, left=0, hspace=0, wspace=0) 
    axes.imshow(imgarr, cmap='gray_r')
    axes.axis('off')
    plt.savefig(filename, dpi=1, format='png') 
    plt.close(fig)

# experiment 1: timestep vs RT

In [3]:
import gc
#############################
# test a single model, and visualize outputs
#############################
task='mnist_c_original'
# task='mnist_c_original'
# task='mnist_recon'
train=False #train or test dataset
print_args=False


# load model
args_to_update = {'device':DEVICE, 'batch_size':BATCHSIZE, 
                 'time_steps': 4, 'routings': 3,'mask_threshold': 0.1}

load_model_path = './models/rrcapsnet/rrcapsnet_best.pt'


args = load_args(load_model_path, args_to_update, print_args)
model = load_model(args)

path_save = './stimuli/stimuli-exp1-step4/'
#  obtain model prediction

d_triallist = {}

for corruption_index in [13, 15]:

    corruption =CORRUPTION_TYPES[corruption_index-1]
    
    d_triallist[corruption] = {}
    
    if task == 'mnist_c_mini':
        x, gtx, y_hot, acc_model, objcaps_len_step, x_recon_step, outputs_model  = \
        evaluate_model_on_mnistc_mini(corruption, model, args, train, verbose=False, save_hooks=True, max_batch_num=None)
        print(f'==> corruption type: {corruption}, this batch acc: {acc_model.mean().item()}')
    elif task =='mnist_c_original':
        print("original is used")
        x, gtx, y_hot, acc_model, objcaps_len_step, x_recon_step = \
        evaluate_model_on_mnistc_original(corruption, model, verbose=False, save_hooks=False)
        print(f'==> corruption type: {corruption}, this batch acc: {acc_model.mean().item()}')
    else:
        batchnum=3*int(1000/BATCHSIZE)-1
        x, gtx, y, objcaps_len_step, x_recon_step, outputs = evaluate_on_batch(task, batchnum, model, args, train)

    # get model prediction
    objcaps_len_step_narrow = objcaps_len_step.narrow(dim=2,start=0, length=args.num_classes)
    # pred_model = objcaps_len_step_narrow.max(dim=-1)[1][:,-1] #torch.Size([1000, 3])


    def get_nstep(objcaps_len_step_narrow, y_hot):
        def get_first_zero_index(x, axis=1):
            cond = (x == 0)
            return ((cond.cumsum(axis) == 1) & cond).max(axis, keepdim=True)[1]

        pstep = objcaps_len_step_narrow.max(dim=-1)[1]
        pnow = pstep[:,1:]
        pbefore = pstep[:,:-1]

        pdiff = (pnow-pbefore)
        null_column = -99*torch.ones(pdiff.size(0),1).to(pdiff.device)
        pdiff = torch.cat([null_column, pdiff], dim=1)
        pdiff[:,-1]=0
        nstep = get_first_zero_index(pdiff)
        pred_model= torch.gather(pstep, 1, nstep).flatten()
        acc_model = torch.eq(pred_model.cpu(), y_hot.max(dim=1)[1])
        return (nstep.flatten()+1).cpu().numpy(), pred_model, acc_model

    nstep, pred_model, acc_model_check = get_nstep(objcaps_len_step_narrow, y_hot)
    assert round(acc_model.mean().item(), 4) == round(acc_model_check.float().mean().item(), 4)


    # get trial id that requires 2 steps and 3 steps among correct trials
    nstep_masked = acc_model.cpu().numpy()*nstep
    id_incorrect = list(np.where(nstep_masked==0)[0])
    id_correct = list(np.where(nstep_masked!=0)[0])    
    print('correct: ', len(id_correct) )
    print('incorrect: ', len(id_incorrect) )

    id_step2 = list(np.where(nstep_masked==2)[0])
    id_step3 = list(np.where(nstep_masked==3)[0])
    id_step4 = list(np.where(nstep_masked==4)[0])
    id_step34 = id_step3 + id_step4
    
    print('step2: ', len(id_step2))
    print('step3: ', len(id_step3))
    print('step4: ', len(id_step4))

    # save tiralist to dictionary
    d_triallist[corruption]['step2'] = id_step2
    d_triallist[corruption]['step3'] = id_step3
    d_triallist[corruption]['step4'] = id_step4
    d_triallist[corruption]['correct'] = id_correct
    d_triallist[corruption]['incorrect'] = id_incorrect

    # get 20 randomly sampled image from each and save
    MAX_TRIAL = 20
    id_step2_sampled = random.sample(id_step2, min(len(id_step2),MAX_TRIAL))
    id_step34_sampled = random.sample(id_step34, min(len(id_step34), MAX_TRIAL))
                                
    for step in ['step2', 'step3', 'step4']: ## ['step2', 'step3', 'step4']
        if step=='step2':
            trialid_to_visualize = id_step2_sampled 
        elif step=='step3':
            trialid_to_visualize = [ti for ti in id_step3 if ti in id_step34_sampled]
        elif step=='step4':
            trialid_to_visualize = [ti for ti in id_step4 if ti in id_step34_sampled]
        
        for trialid in trialid_to_visualize:
            our_pred = pred_model[trialid].cpu().item()
            gt =y_hot.max(dim=1)[1][trialid].cpu().item()

            # save image x8 original pixel size
            imgarray = x[trialid].numpy()
            filename = f'{corruption}_{step}_t{trialid}_g{gt}_o{our_pred}.png'
            save_imgarr(np.transpose(imgarray,(1,2,0)), filename= path_save+filename)
            
    print('all images are saved')
    gc.collect()
    
    
# ==> corruption type: shot_noise, this batch acc: 0.9418999552726746
# incorrect:  581
# step2:  9380
# step3:  22
# step4:  17


# ==> corruption type: impulse_noise, this batch acc: 0.9120000004768372
# incorrect:  880
# step2:  7934
# step3:  1089
# step4:  97

# ==> corruption type: glass_blur, this batch acc: 0.8628000020980835
# incorrect:  1372
# step2:  8387
# step3:  201
# step4:  40

# ==> corruption type: motion_blur, this batch acc: 0.936199963092804
# incorrect:  638
# step2:  9297
# step3:  51
# step4:  14

# ==> corruption type: shear, this batch acc: 0.9767999649047852
# incorrect:  232
# step2:  9742
# step3:  26
# step4:  0

# ==> corruption type: scale, this batch acc: 0.9440999627113342
# incorrect:  559
# step2:  9432
# step3:  3
# step4:  6

# ==> corruption type: rotate, this batch acc: 0.9200999736785889
# incorrect:  799
# step2:  9162
# step3:  34
# step4:  5

# ==> corruption type: brightness, this batch acc: 0.9892999529838562
# incorrect:  107
# step2:  9890
# step3:  3
# step4:  0

# ==> corruption type: translate, this batch acc: 0.47119998931884766
# incorrect:  5288
# step2:  4577
# step3:  101
# step4:  34

# ==> corruption type: stripe, this batch acc: 0.836899995803833
# incorrect:  1631
# step2:  8205
# step3:  154
# step4:  10

# ==> corruption type: fog, this batch acc: 0.8615999817848206
# incorrect:  1384
# step2:  5842
# step3:  2715
# step4:  59

# ==> corruption type: spatter, this batch acc: 0.9839999675750732
# incorrect:  160
# step2:  9794
# step3:  41
# step4:  5

    
# ==> corruption type: dotted_line, this batch acc: 0.983299970626831
# incorrect:  167
# step2:  9777
# step3:  53
# step4:  3

# ==> corruption type: zigzag, this batch acc: 0.952299952507019
# incorrect:  477
# step2:  9395
# step3:  118
# step4:  10

# ==> corruption type: canny_edges, this batch acc: 0.6972999572753906
# incorrect:  3027
# step2:  6704
# step3:  224
# step4:  45


=========== model instantiated like below: =============
TASK: mnist_recon (# targets: 1, # classes: 10, # background: 0)
TIMESTEPS #: 4
ENCODER: two-conv-layer w/ None projection
...resulting primary caps #: 1152, dim: 8
ROUTINGS # 3
Object #: 10, BG Capsule #: 0
DECODER: fcn, w/ None projection
...recon only one object capsule: True
...use recon mask for attention: True
...with mask type bool, threshold 0.1, apply_method match

original is used
==> corruption type: spatter, this batch acc: 0.9830999970436096
correct:  9831
incorrect:  169
step2:  9768
step3:  59
step4:  4
all images are saved
original is used
==> corruption type: zigzag, this batch acc: 0.9555999636650085
correct:  9556
incorrect:  444
step2:  9406
step3:  132
step4:  18
all images are saved


In [14]:
## change filenames
# import os
# path = './stimuli/stimuli-exp1-step4/'

# for f in os.listdir(path):
#     if not f.startswith('.'):
#         fsplit = f.split('_')
#         stepsize = fsplit[-1].split('.')[0]
#         corruption =  fsplit[:-4]
#         newf = '_'.join(corruption + [stepsize] + fsplit[-4:-1] ) + '.png'
#         os.rename(path+f, path+newf)

# experiment 2: compare with cnn

In [6]:
#############################
# test a single model, and visualize outputs
#############################
task='mnist_c_original'
# task='mnist_c_original'
# task='mnist_recon'
train=False #train or test dataset
print_args=False


# load our model
args_to_update = {'device':DEVICE, 'batch_size':BATCHSIZE, 
                 'time_steps': 4, 'routings': 3, 'routing_type': 'pd-recon', #'original' 
                 'min_coup': 0.5, 'min_rscore': 0.5, 'mask_threshold': 0.1}
# load_model_path ='./results/mnist/Mar21_0248_run3/archive_model_epoch50_acc0.9907.pt'
load_model_path = './models/rrcapsnet/rrcapsnet_best.pt'

args = load_args(load_model_path, args_to_update, print_args)
model = load_model(args)

# load cnn
from train_cnn import *
path_cnn = './models/cnn/cnn_best.pt'
cnn = Net().to(DEVICE)
cnn.load_state_dict(torch.load(path_cnn))
cnn.eval()

path_save = './stimuli/stimuli-exp2-step4/'



# obtain model predictions

for corruption_index in [4, 9]:

    corruption =CORRUPTION_TYPES[corruption_index-1]
    

    if task == 'mnist_c_mini':
        x, gtx, y_hot, acc_model, objcaps_len_step, x_recon_step, outputs_model  = \
        evaluate_model_on_mnistc_mini(corruption, model, args, train, verbose=False, save_hooks=True, max_batch_num=None)
        print(f'==> corruption type: {corruption}, this batch acc: {acc_model.mean().item()}')
    elif task =='mnist_c_original':
        print("original is used")
        x, gtx, y_hot, acc_model, objcaps_len_step, x_recon_step = \
        evaluate_model_on_mnistc_original(corruption, model, verbose=False, save_hooks=False)
        print(f'==> corruption type: {corruption}, this batch acc: {acc_model.mean().item()}')
    else:
        batchnum=3*int(1000/BATCHSIZE)-1
        x, gtx, y, objcaps_len_step, x_recon_step, outputs = evaluate_on_batch(task, batchnum, model, args, train)

    # get model prediction
    objcaps_len_step_narrow = objcaps_len_step.narrow(dim=2,start=0, length=args.num_classes)
    # pred_model = objcaps_len_step_narrow.max(dim=-1)[1][:,-1] #torch.Size([1000, 3])


    def get_nstep(objcaps_len_step_narrow, y_hot):
        def get_first_zero_index(x, axis=1):
            cond = (x == 0)
            return ((cond.cumsum(axis) == 1) & cond).max(axis, keepdim=True)[1]

        pstep = objcaps_len_step_narrow.max(dim=-1)[1]
        pnow = pstep[:,1:]
        pbefore = pstep[:,:-1]

        pdiff = (pnow-pbefore)
        null_column = -99*torch.ones(pdiff.size(0),1).to(pdiff.device)
        pdiff = torch.cat([null_column, pdiff], dim=1)
        pdiff[:,-1]=0
        nstep = get_first_zero_index(pdiff)
        pred_model= torch.gather(pstep, 1, nstep).flatten()
        acc_model = torch.eq(pred_model.cpu(), y_hot.max(dim=1)[1])
        return (nstep.flatten()+1).cpu().numpy(), pred_model, acc_model

    nstep, pred_model, acc_model_check = get_nstep(objcaps_len_step_narrow, y_hot)
    assert round(acc_model.mean().item(), 4) == round(acc_model_check.float().mean().item(), 4)


    ##################
    # get cnn prediction
    ##################
    if task == 'mnist_c_mini':
        data_cnn, target_cnn, logsoft_cnn, pred_cnn, acc_cnn \
        = evaluate_cnn_on_mnistc_mini(corruption, cnn, max_batch_num=None)
        print(f'==> corruption type: {corruption}, this batch acc: {acc_cnn.float().mean().item()}')
    elif task =='mnist_c_original':
        print("original is used")
        data_cnn, target_cnn, logsoft_cnn, pred_cnn, acc_cnn \
        =  evaluate_cnn_on_mnistc_original(corruption, cnn)
        print(f'==> corruption type: {corruption}, this batch acc: {acc_cnn.float().mean().item()}')
        
        
    #######################
    # get trials id where both model fails & disagree
    #######################
    assert (target_cnn.cpu() == y_hot.max(dim=1)[1].cpu()).all()
    bool_bothincorrect = ~(acc_model.bool())* ~(acc_cnn.bool())
    bool_diffanswer = (pred_model!= pred_cnn)
    bool_onlycnncorrect = ~(acc_model.bool())* (acc_cnn.bool())
    # idx_bothincorrect = torch.nonzero(bool_bothincorrect)
    trialid_interest = torch.nonzero(bool_bothincorrect*bool_diffanswer).flatten().tolist()
    # trialid_interest = torch.nonzero(bool_onlycnncorrect*bool_diffanswer).flatten().tolist()
    print(len(trialid_interest))
    
    #####################
    # save image (in reverted grayscale) and trialinfo
    #####################

    d = collections.defaultdict(dict)
    MAX_TRIAL = 20

    trialid_to_visualize = random.sample(trialid_interest, min(len(trialid_interest), MAX_TRIAL))

    for trialid in trialid_to_visualize:
        cnn_pred = pred_cnn[trialid].cpu().item()
        our_pred = pred_model[trialid].cpu().item()
        gt =target_cnn[trialid].cpu().item()

        # save image x8 original pixel size
        imgarray = x[trialid].numpy()
        filename = f'{corruption}_t{trialid}_g{gt}_c{cnn_pred}_o{our_pred}.png'
        save_imgarr(np.transpose(imgarray,(1,2,0)), filename=path_save+filename)

        # save trial info to dictionary
        d[filename]['id'] = trialid
        d[filename]['imgarray'] = imgarray.tolist()
        d[filename]['cnn_softmax'] = torch.exp(logsoft_cnn[trialid]).cpu().numpy().tolist()
        d[filename]['our_objlen'] = objcaps_len_step_narrow[trialid, -1].cpu().numpy().tolist()
        d[filename]['cnn_pred'] = cnn_pred
        d[filename]['our_pred'] = our_pred
        d[filename]['gt'] =  gt


    # save dictionary
    with open(path_save+ f'{corruption}.json', 'w') as fp:
        json.dump(d, fp)
    print('trialinfo saved to disk!')    
    # to open
    # with open('./stimuli/glass_blur.json') as json_file:
    #     test = json.load(json_file)


=========== model instantiated like below: =============
TASK: mnist_recon (# targets: 1, # classes: 10, # background: 0)
TIMESTEPS #: 4
ENCODER: two-conv-layer w/ None projection
...resulting primary caps #: 1152, dim: 8
ROUTINGS # 3
Object #: 10, BG Capsule #: 0
DECODER: fcn, w/ None projection
...recon only one object capsule: True
...use recon mask for attention: True
...with mask type bool, threshold 0.1, apply_method match

original is used
==> corruption type: glass_blur, this batch acc: 0.8671000003814697
original is used
==> corruption type: glass_blur, this batch acc: 0.9012999534606934
178
trialinfo saved to disk!
original is used
==> corruption type: brightness, this batch acc: 0.9884999990463257
original is used
==> corruption type: brightness, this batch acc: 0.876800000667572
30
trialinfo saved to disk!
